In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import sys
sys.path.append("../")
from PythonCode.preprocess.common import *
from PythonCode.preprocess.complexStyleFeatures import *
from PythonCode.preprocess.simpleStyleFeatures import *

In [3]:
df = load_data("../Data/C50/C50test", 50)

In [4]:
df

,author_name,book_name,book_text
0,AaronPressman,421829newsML.txt,U.S. Senators on Tuesday sharply criticized a ...
1,AaronPressman,424074newsML.txt,Two members of Congress criticised the Federal...
2,AaronPressman,42764newsML.txt,Commuters stuck in traffic on the Leesburg Pik...
3,AaronPressman,43033newsML.txt,A broad coalition of corporations went to Capi...
4,AaronPressman,433558newsML.txt,"On the Internet, where new products come and g..."
...,...,...,...
2495,WilliamKazer,504283newsML.txt,China has scored new successes in its fight ag...
2496,WilliamKazer,504526newsML.txt,China has scored new successes in its fight ag...
2497,WilliamKazer,51502newsML.txt,China is on target with plans to to promote 10...
2498,WilliamKazer,522090newsML.txt,China may need to adjust the mix of its treasu...


In [6]:
from transformers import BertTokenizer, TFBertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = TFBertModel.from_pretrained("bert-base-cased")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='tf')
output = model(encoded_input)

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [6]:
import tensorflow as tf

def preprocess(x):
    return text_vectorizer(x)

def forward_pass(x):
    return tf.keras.layers.Dense(1)(x)  # Linear model

inputs = tf.keras.Input(shape=(1,), dtype='string')
outputs = forward_pass(preprocess(inputs))
model = tf.keras.Model(inputs, outputs)
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True))
model.fit(train_ds, epochs=5)

NameError: name 'text_vectorizer' is not defined

In [9]:
df = merge_datasets()

In [14]:
type(df[TEXT_COLUMN_NAME].values[0])

str

<!-- https://huggingface.co/dslim/bert-base-NER -->

In [15]:
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from src.config.Constants import *
from keras.layers import Dense, GRU, AvgPool1D, Masking

from src.preprocess.common import merge_datasets, num_sentences_based_chucking
from src.preprocess.word_embedding_features import sentence_level_preprocess

DATA_PATH = "../data/C50"


def article_level_pipeline():
    def forward_pass(x):
        x = Masking(mask_value=0., input_shape=(MAX_NUMBER_OF_SENTENCE, MAX_SENTENCE_LENGTH))(x)
        x = GRU(100, recurrent_dropout=0.2, return_sequences=True)(x)
        x = AvgPool1D(pool_size=(MAX_NUMBER_OF_SENTENCE,))(x)
        return Dense(50, activation="softmax")(x)

    X_train, X_test, y_train, y_test = train_test_split(df[TEXT_COLUMN_NAME], df[AUTHOR_NAME_COLUMN_NAME],
                                                        test_size=TEST_PART)
    inputs = tf.keras.Input(shape=(1,), dtype='string')
    outputs = forward_pass(sentence_level_preprocess(inputs))
    model = tf.keras.Model(inputs, outputs)
    model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])
    print(model.summary())
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=VALIDATION_PART)

    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30)
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=f"./article-level-checkpoints",
                                                                   save_weights_only=False,
                                                                   monitor='val_accuracy', mode='max',
                                                                   save_best_only=True)
    history = model.fit(x=X_train, y=y_train, epochs=30000, shuffle=True,
                        batch_size=200, validation_data=(X_val, y_val), callbacks=[callback, model_checkpoint_callback])
    with open("article-level-history", "w") as file:
        pickle.dump(history, file)
    model.save("article-level")


def sentence_level_pipeline(df: pd.DataFrame):
    def forward_pass(x):
        x = Masking(mask_value=0., input_shape=(MAX_LENGTH, EMBEDDING_SIZE))(x)
        x = GRU(100, recurrent_dropout=0.2, return_sequences=True)(x)
        x = AvgPool1D(pool_size=(170,))(x)
        return Dense(50, activation="softmax")(x)

    X = df[TEXT_COLUMN_NAME]
    data = num_sentences_based_chucking(X, NUM_OF_SENTENCE_CHUNK, "X")
    X_train, X_test, y_train, y_test = train_test_split(data["X"], data["y"], test_size=TEST_PART)
    inputs = tf.keras.Input(shape=(1,), dtype='string')
    outputs = forward_pass(sentence_level_preprocess(inputs))
    model = tf.keras.Model(inputs, outputs)
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True))
    model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])
    print(model.summary())
    return model


sentence_level_pipeline(merge_datasets())

TypeError: num_sentences_based_chucking() takes 2 positional arguments but 3 were given